# Week 6, Lesson 4, Activity 6: Simple LM algorithm

&copy;2021, Ekaterina Kochmar \
(revised: Nadejda Roubtsova, June 2022)

Your task in this activity is to:

- Implement and evaluate a simple language model using the data referenced in the notebook.

## Step 1: Define a Language Model class

Let's start by importing the libraries and building a Language Model class:

In [1]:
from collections import defaultdict, Counter
from numpy import cumsum, sum, searchsorted
from numpy.random import rand

The objects of class `LanguageModel` should be able to:

* _estimate_ the transition probabilities from the context of given order (i.e., length of context) to the next characters based on some training text;
* _predict_ the next character based on any new context;
* _generate_ a whole sequence using its predictions.

To this end let's add three public methods to the class `LanguageModel` that will provide all the functionality described above:

* `train` to estimate the probabilities;
* `predict` to predict next character;
* `generate` to generate a whole sequence of characters.

In the code below:
1. The method `train` learns the transition probabilities from a text, which is represented as a string and provided to the method as an argument `sequence`.
2. The `transitions` dictionary keeps the number of times the context of the specified order (length) is followed by the current character.
3. The method `predict` chooses the most probable character given the preceding one(s). The preceding one(s) are provided to the method as an argument `symbol`.
4. If the length of the provided sequence of the previous characters doesn't match the language model order, report an error.
5. Return the character with a given probability.
6. The method `generate` allows you to generate a sequence of a specified number (`n`) of characters.
7. For that, it calls on the `predict` method providing it with the context `start`.
8. It moves the context character by character specified number of `n` times, thus allowing you to generate `n` new characters.
9. Method `weighted_pick` provides search functionality for the probabilities.

Your task is to read through the code, experiment with it, and make sure you understand the process.

In [2]:
class LanguageModel(object):
    def __init__(self, order=1):
        '''Initializes a language model of the given order.'''
        self._transitions = defaultdict(int)
        self._order = order
        
    def train(self, sequence):
        '''Trains the model using sequence.'''
        self._symbols = list(set(sequence))
        for i in range(len(sequence)-self._order):
            self._transitions[sequence[i:i+self._order], sequence[i+self._order]] += 1

    def predict(self, symbol):
        '''Takes as input a string and predicts the next character.'''
        if len(symbol) != self._order:
            raise ValueError('Expected string of %d chars, got %d' % (self._order, len(symbol)))
        probs = [self._transitions[(symbol, s)] for s in self._symbols]
        return self._symbols[self._weighted_pick(probs)]

    def generate(self, start, n):
        '''Generates n characters from start.'''
        result = start
        for i in range(n):
            new = self.predict(start)
            result += new
            start = start[1:] + new
        return result

    @staticmethod
    def _weighted_pick(weights):
        '''Weighted random selection returns n_picks random indexes.
        The chance to pick the index i is given by weights[i].'''
        return searchsorted(cumsum(weights), rand()*sum(weights))

## Step 2: Use your Language Model in practice

Let's try to generate texts using famous books to train the language model. In that case, you should expect to get the generated text that is quite similar in style to the text of the book you trained your language model on.

Let's import [urllib](https://docs.python.org/3.0/library/urllib.request.html) that will allow you to access texts from online resources. You can download a book from a collection of [Gutenberg Project books](https://www.gutenberg.org) available online. Let's start with *Robinson Crusoe*:

In [3]:
from urllib.request import urlopen

in_text = ""
with urlopen('https://www.gutenberg.org/files/521/521-0.txt') as response:
    for line in response:
        line = line.decode('utf-8')  # Decoding the binary data to text.
        in_text += line
        
print(type(in_text))
print(len(in_text))
print(in_text[:75])

<class 'str'>
651508
﻿The Project Gutenberg eBook of The Life and Adventures of Robinson Crusoe,


This piece of code downloads Robinson Crusoe and puts all the contents of this book in a (very long) string.
Let's now train your language model and generate new text, for example, using a context of 4 previous characters:

In [4]:
model = LanguageModel(order=4)
model.train(in_text)
# Note that to generate text with a context (order) of specific length,
# you need to provide the context of that length: e.g., len('your')=4 here
print (model.generate('your', 100))

your monstances more of ours, we tree of me in my devils, so overnor’s sands and
land, and but we
with


Experiment with examples of texts of your own choice. E.g., here's Shakespeare:

In [5]:
book = 'http://www.gutenberg.org/cache/epub/1112/pg1112.txt' # Romeo and Juliet

# use the code similar to the steps above

## Step 3: Evaluate your Language Model

The most widely used measure of the language model performance is *perplexity*, which measures how probable in the language the piece of text generated by a language model is. Let's implement this measure and evaluate the text generated by the language models above.

The first step is to collect the data which you will use to calculate probabilities. You can use some text from before as the data for perplexity estimation. 

The data contains strings of symbols (characters) and before calculating the probabilities of words, you need to tokenise it into constituent words. Let's use `spaCy` for that:

In [6]:
import spacy
nlp = spacy.load('en_core_web_md')
tokens = nlp(in_text)
print(len(tokens))

OSError: [E050] Can't find model 'en_core_web_md'. It doesn't seem to be a Python package or a valid path to a data directory.

Let's estimate unigram (word-based) probabilities in this data:

In [ ]:
def unigram(tokens):    
    model = defaultdict(lambda: 0.001)
    for token in tokens:
        token = str(token).strip()
        model[token] += 1
       
    total = 0
    # calculate the total number of occurrences of all words and store as total
    
    for word in model:
        model[word] = model.get(word)/float(total)
    return model

Now, let's estimate perplexity using the formula from the lecrtures:

In [ ]:
def perplexity(testset, model):
    testset = nlp(testset)
    perplexity = 1
    N = 0
    for word in testset:
        N += 1
        perplexity = perplexity * (1/model[str(word).strip()])
    perplexity = pow(perplexity, 1/float(N)) 
    return perplexity

Note, that a good language model will generate a highly probable sequence of words. Recall from the lecture that lower perplexity values signify better models. When you use perplexity to measure the quality of and to compare several language models, you are looking for the one that has **lower perplexity**.

With all the components in place, let's apply this measurement and compare a language model that generates text using previous $4$ characters with the one that uses previous $6$ characters. The more likely the words generated by the language model are, the lower the perplexity score. Which model is better?

In [ ]:
lm_data = in_text
model = unigram(tokens)

lm1 = # initialise a language model of order=4
lm1.train(lm_data)
testset1 = # generate some text for testset1 using lm1
print(testset1)
print(# calculate perplexity for this text according to the model
      )

print()

lm2 = # initialise a language model of order=6
lm2.train(lm_data)
testset2 = # generate some text for testset1 using lm2
print(testset2)
print(# calculate perplexity for this text according to the model
      )